<a href="https://colab.research.google.com/github/TheOneTrueGuy/DigitalDayDream/blob/main/Copy_of_FarStrike_Sonnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# What do these installs tell us about the code we are going to be writing?
# How will the functions and operation depend on these dependencies?
# What methods, variables, objects and function from these libraries will be used to achieve these ends?
# watch closely for the imports and import from statements that call on these libraries.
!pip install git+https://github.com/huggingface/diffusers.git
!pip install --upgrade accelerate #transformers
!pip install omegaconf
!apt-get install rsync # for better file transfer progress?
!pip install noise
!pip install anthropic # for Claude Sonnet
# These libraries will be used to generate image-to-image diffusion model outputs specifically an image.
# More specifically, diffusers is a library for training diffusion models, generating images
# accelerate is a library for distributed inference, and transformers is a library for natural language processing.
# and this package (notebook) as a whole is used to generate frames of an animation using diffusion models, huggingface pipelines and imagemagick convert command


In [ ]:
!git lfs install
!git clone https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers
#

In [ ]:
#!pip install omegaconf #for use with deliberate
from google.colab import drive
drive.mount('/content/drive')

#!mkdir 8_1_2023
!mkdir src

#!cp /content/drive/MyDrive/stub_materials/p17.png  /content/st.png # cosmicconnection
!cp /content/drive/MyDrive/stub_materials/bio10_12_23.txt /content/
!cp /content/drive/MyDrive/stub_materials/building.txt /content/


In [ ]:
!apt-get update
!apt install imagemagick


In [ ]:
#@title upload start image 512x512 will be quick and look good

import os

from google.colab import files
uploaded = files.upload()
name=uploaded.keys()
name = list(name)[0]
print(name)


In [ ]:

os.system(f"convert {name} -resize 512x512! st.png")


In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="your anthropic key here",
)

def generate_text(prompt):
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    print(message.content)
    return message.content



In [ ]:
import anthropic
from PIL import Image
import base64
import io

def load_and_encode_image(image_path, media_type):
    with Image.open(image_path) as img:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=media_type.split('/')[-1].upper())
        img_byte_arr = img_byte_arr.getvalue()
        image_data = base64.b64encode(img_byte_arr).decode("utf-8")
    return {
        "data": image_data,
        "media_type": media_type
    }

def create_image_message(image_path, text_prompt):
    # Determine media type based on file extension
    media_type = f"image/{image_path.split('.')[-1].lower()}"
    if media_type == "image/jpg":
        media_type = "image/jpeg"

    # Load and encode the image
    image_info = load_and_encode_image(image_path, media_type)

    # Set up the Anthropic client
    client = anthropic.Anthropic()

    # Create and return the message
    return client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": image_info['media_type'],
                            "data": image_info['data'],
                        },
                    },
                    {
                        "type": "text",
                        "text": text_prompt
                    }
                ],
            }
        ],
    )



# Example usage
image_path = "st.png"
text_prompt = "Describe this image."

response = create_image_message(image_path, text_prompt)
print(response)

def image_to_message(img: Image.Image, text_prompt: str, media_type: str = None):
    def encode_image(img: Image.Image, media_type: str):
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=media_type.split('/')[-1].upper())
        img_byte_arr = img_byte_arr.getvalue()
        image_data = base64.b64encode(img_byte_arr).decode("utf-8")
        return {
            "data": image_data,
            "media_type": media_type
        }

    # Determine media type if not provided
    if media_type is None:
        if img.format == 'PNG':
            media_type = 'image/png'
        elif img.format in ['JPEG', 'JPG']:
            media_type = 'image/jpeg'
        else:
            media_type = f'image/{img.format.lower()}'

    # Encode the image
    image_info = encode_image(img, media_type)

    # Set up the Anthropic client
    client = anthropic.Anthropic()

    # Create and return the message
    return client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": image_info['media_type'],
                            "data": image_info['data'],
                        },
                    },
                    {
                        "type": "text",
                        "text": text_prompt
                    }
                ],
            }
        ],
    )

In [ ]:
# run this first then make anim in next cell

import torch

from diffusers.utils import load_image
from transformers import AutoTokenizer # for use with small sd

import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image, ImageOps, ImageChops, ImageFilter

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, [False])

torch.cuda.empty_cache()

hf_auth="huggingface auth key here"

from diffusers import StableDiffusion3Img2ImgPipeline


pipe = StableDiffusion3Img2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers",
                                                torch_dtype=torch.float16, token=hf_auth,  text_encoder_3=None, tokenizer_3=None)

pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()


In [ ]:
import string #cell number 11!
# subject description, background, additional elements, effect, overall composition

# how about trying this with a procedural method loop?

zip_name = "06_25_24_farstrike_sonnet.zip"  #@param {type:"string"}

outdir = dt.datetime.now().strftime("%Y%m%d%H%M")
# if outdir exists add 4 random digits to it:
if os.path.exists(outdir):
    # Generate 4 random digits
    random_digits = str(random.randint(1000, 9999))

    # Create a new output directory with the random digits appended
    new_outdir = outdir + random_digits

    os.mkdir(new_outdir)
    outdir=new_outdir
    print("Created new output directory:", new_outdir)
else:
    os.system(f"mkdir {outdir}")
print("date time now:" + outdir)

os.system(f"cp st.png {outdir}")
style="" # can add style to prompt later

xl=0.31
xr=0
gs=7.21
gd=.1
ns=0.65
xz=0
nx=0


total=200
xx = random.randint(2, 510)
yy = random.randint(2, 510)
tx = random.choice(string.ascii_lowercase + string.ascii_uppercase)

negprompt=""

initial_value = 1  # Initial value of C2 use for exact-ish file count and timing



image3= load_image("st.png").convert("RGB")
for xn in range(0,total,1):#must equal xz


  if xn%50==49: torch.cuda.empty_cache()

  negstyle=""
  negprompt=""
  #negstyle="dumb, flat, ordinary, monochrome, boring, sad, insipid, ordinary, human face" # , unhappy, worried, angry , weird face, scarlet, distorted face, weird nose" # human, person, face, head, hands, man, woman, child, " # dispirated, terrorized,  malformed hands, misshapen hands,
  negprompt = negprompt + negstyle #prefix with style
  print(prompt)

  init_image = load_image("st.png").convert("RGB")

  instruction="""
  This image is a frame in an animation. The animation subject is nanotechnology. Using the formula:
   subject description, background, additional elements, effect, overall composition
   you will suggest a brief image prompt that would change this image into the next frame in the animation. Reply only with:
   subject description, background, additional elements, effect, overall composition
  """
  prompt=image_to_message(init_image, instruction)

  nsize=(512,512)
  init_image=init_image.resize(nsize)
  print("negative:"+negprompt)

  image2 = init_image
  if xn>2:
    image2=stacky.get_item(2)

  image = pipe(
               prompt,
               num_inference_steps=28,
               strength=0.5,
               guidance_scale=gs, #11.721,
               negative_prompt = negprompt,
               image=init_image, # init_image,
               ).images[0]
  gs+=.006  #sr=sin_ratio(xn, 0.086)



  stacky.push(image)
  outname= "st.png"

  out2=outdir+"/st-" + str(xz).zfill(4) + ".png"
  image.save(out2) # save animation frame
  xz+=1
  #out2=outdir+"/st-" + str(xz).zfill(4) + ".png" # 3 lines added 2/14/24 to roll back remove/comment and change image.save above to imageb.save
  #imageb.save(out2)
  #xz+=1


  image.save(outname)

os.system(f"cp 1julien3.sh {outdir}")
#os.system(f"cp moto1g.txt {outdir}")
os.system(f"zip -r {zip_name} {outdir}")
os.system(f"cp {zip_name}  /content/drive/MyDrive/product")


In [ ]:
# Guy - don't you dare erase that!
# split mode for top/bottom seperate motion
def split_mode(img):
  pass


# New Section

In [ ]:
!zip -r 8_2_2023_farstrike2c.zip 20230802/
!cp 8_2_2023_farstrike2c.zip  /content/drive/MyDrive/product

In [ ]:
os.system(f"zip -r {zip_name} {outdir}")
os.system(f"cp {zip_name}  /content/drive/MyDrive/product")

In [ ]:
print(zip_name)

In [ ]:
#run this cell after starting cell above if you don't want to burn
#google compute credits unnecessarily.
print(dt.datetime.now().strftime("%Y%M%d%h%m"))
from google.colab import runtime
runtime.unassign()

In [ ]:
xx = random.randint(2, 510)
yy = random.randint(2, 510)
tx = random.choice(string.ascii_lowercase + string.ascii_uppercase)
init_image = load_image("st.png").convert("RGB")
init_image=putText(tx, init_image, xx, yy, 84, 0, 0, 0)
init_image.save('st.png')
init_image

In [ ]:
file = open("bio10_12_23.txt") #be double sure to scrub your input
line = file.read().replace("\n", " ")
line = line.replace("   ", " ") # replace triple spaces
line = line.replace("  ", " ") # replace double spaces
file.close()
words=line.split(" ")
lenwo=len(words)
print(lenwo)

In [ ]:
!pip install omegaconf

In [ ]:

import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
import datetime as dt
import time
import os
import random


class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)

#if pipe is None:
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16
)
#pipe = pipe.to("cuda")
#pipe.safety_checker = FakeSafety()
#excise all above after 8/3/2023 so that previous cell does all the pipe creation and handling

#url = "st-0027.png"
#"https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/aa_xl/000000009.png"
# outdir should be a string of current date



In [ ]:
import requests
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"

#response = requests.get(url)
#init_image = Image.open("51P90MVCr8L._AC_UF894,1000_QL80_.jpg").convert("RGB")
init_image = Image.open("fantasy_landscape.png").convert("RGB")

#init_image = Image.open(BytesIO(response.content)).convert("RGB")
#init_image = init_image.resize((768, 512))

#prompt = "Glossy movie poster, high quality, Kevin Sorbo in the live action Darkwing Duck remake with Keanu Reeves"


images = pipe(prompt=prompt, num_inference_steps=122, image=init_image, strength=0.95, guidance_scale=13.5).images
images[0].save("fantasy_landscape2.png")

In [ ]:
# pallette controls
# convert input.png mask.png -fx 'u.r, u.g*1.5, u.b' output.png
# convert input.png -separate -channel G -evaluate Multiply 1.5 +channel output.png
# convert input.png mask.png -channel RGB -fx 'u.r + (u.g - u.r) * mask, u.g + (u.g - u.r) * mask, u.b + (u.g - u.r) * mask' output.png
# convert input.png -channel RGB -fx 'clamp(u.r, 20, 70), clamp(u.g, 120, 255), clamp(u.b, 128, 191)' output.png


In [ ]:
!wget -O deliberatexp.safetensors  https://civitai.com/api/download/models/19724?type=Model&format=SafeTensor&size=full&fp=fp16

In [ ]:
# convert img1.png img2.png -fx “(((255u)&(255(1-v)))|((255*(1-u))&(255*v)))/255” img3.png
# convert image1.png image2.png -morphology HMT ‘3x3: 0,1,0 1,1,1 0,1,0’ result.png
# convert image1.png image2.png -fx ‘u-v’ result.png

In [ ]:
#segmind/small-sd
# distilled model
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("segmind/small-sd", torch_dtype=torch.float16)
prompt = "Portrait of a pretty girl"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
image = pipeline(prompt, negative_prompt = negative_prompt).images[0]
image.save("my_image.png")


In [ ]:
from PIL import Image, ImageChops

# Open the two images you want to compare
image1 = Image.open("image1.jpg")
image2 = Image.open("image2.jpg")

# Ensure that both images have the same size
image2 = image2.resize(image1.size)

# Calculate the absolute difference between the two images
diff = ImageChops.difference(image1, image2)

# You can use XOR instead of difference if you prefer
# diff = ImageChops.logical_xor(image1, image2)

# Convert the difference image to grayscale
diff = diff.convert('L')

# Threshold the difference image to create a mask
threshold = 50  # You can adjust this threshold based on your needs
mask = diff.point(lambda p: p > threshold and 255)

# Save the mask image
mask.save("difference_mask.png")


In [ ]:
from PIL import Image, ImageChops, ImageOps

# Open the two images you want to compare
image1 = Image.open("image1.jpg")
image2 = Image.open("image2.jpg")

# Ensure that both images have the same size
image2 = image2.resize(image1.size)

# Calculate the absolute difference between the two images
diff = ImageChops.difference(image1, image2)

# Convert the difference image to grayscale
diff = diff.convert('L')

# Threshold the difference image to create a mask
threshold = 50  # You can adjust this threshold based on your needs
mask = diff.point(lambda p: p > threshold and 255)

# Invert the mask
inverted_mask = ImageOps.invert(mask)

# Invert the difference directly
inverted_diff = ImageOps.invert(diff)

# Blend the inverted difference back into the original image
blend_factor = 0.1  # You can adjust this factor based on how much you want to reduce the difference
result_image = Image.blend(image1, image2, blend_factor, inverted_diff)

# Save the result image
result_image.save("result_image.jpg")
